### Engineering

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
output, received, 

In [ ]:
step 1: parititon weights
step 2: mask targets
step 3: calculate local embeddings
step 4: calculate global embeddings

In [ ]:
step 1: 1000/4=250
step 2: 1*250=250

In [ ]:
start_idx+partition_size

In [ ]:
step 1: partition weights
step 2: mask targets
step 3: calculate local predicted logits
step 4: calculate global predicted logits
step 5: log

In [ ]:
step 1: send m

In [ ]:
parititon_size*rank

In [ ]:
gpu 0: [0, 122, 0]
gpu 1: [567, 0, 888]

In [2]:
import torch.distributed as dist
import torch.distributed.rpc as rpc

In [4]:
class Reduce(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, parallel_context):
        group = parallel_context.get_group(ParallelMode.TENSOR)
        dist.all_reduce(input, group=group)
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        return tuple([grad_input, None])

In [5]:
class ParallelEmbedding(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, parallel_context):
        super().__init__()
        world_size = parallel_context.get_world_size(ParallelMode.TENSOR)
        per_partition = num_embeddings // world_size
        
        self.weight = nn.Parameter(torch.randn(
            per_partition,
            embedding_dim
        ))
        self.vocab_start_idx, self.vocab_end_idx = self._get_vocab_range(
            per_partition, parallel_context
        )
        self.parallel_context = parallel_context
    
    def _get_vocab_range(self, per_partition, parallel_context):
        rank = parallel_context.get_local_rank(ParallelMode.TENSOR)
        start_idx = rank*per_partition
        end_idx = start_idx+per_partition
        return start_idx, end_idx

    def forward(self, inputs):
        input_mask = (inputs < self.vocab_start_idx) | (inputs >= self.vocab_end_idx)
        masked_input = inputs.clone() - self.vocab_start_idx
        masked_input[input_mask] = 0
        
        parallel_embeddings = F.embedding(masked_input, self.weight)
        parallel_embeddings[input_mask, :] = 0.
        
        embeddings = Reduce.apply(parallel_embeddings, self.parallel_context)
        return embeddings

In [7]:
class Broadcast(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        dist.all_reduce(grad_input)
        return grad_input

In [8]:
class Gather(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        world_size = dist.get_world_size()
        inputs = [torch.zeros_like(input) for _ in range(world_size)]
        dist.all_gather(inputs, input)
        inputs = torch.cat(inputs)
        return inputs
    
    @staticmethod
    def backward(ctx, grad_input):
        world_size = dist.get_world_size()
        rank = dist.get_rank()
        
        chunks = torch.split(
            grad_input,
            split_size_or_sections=grad_input.shape[-1] // world_size
        )
        return chunks[rank]

In [6]:
class ColumnParallelLinear(nn.Module):
    def __init__(self, input_size, output_size, world_size):
        super().__init__()
        per_partition = output_size // world_size
        
        self.weight = nn.Parameter(torch.randn(
            per_partition,
            input_size
        ))
        self.bias = nn.Parameter(torch.randn(
            per_partition
        ))
    
    def forward(self, input):
        input_parallel = Broadcast.apply(input)
        output_parallel = F.linear(input_parallel, self.weight, self.bias)
        outputs = Gather.apply(output_parallel)
        return outputs

In [ ]:
step 1:

In [ ]:
agent_rref.rpc_sync().your_mom(69)

In [ ]:
no userrrefs, 

In [9]:
from contextlib import contextmanager

In [11]:
@contextmanager
def use_stream(stream):
    if not isinstance(stream, torch.cuda.Stream):
        yield
        return
    
    with torch.cuda.stream(stream):
        yield

In [ ]:
dist.isend()

In [ ]:
no userrrefs references, no referecences in owner node

In [ ]:
step 1: 

In [ ]:
step 1: send metadata
step 2: send raw data
step 3: receiver received data, construct placeholder tensor
step 4: receive the raw data, fill into the placeholder

In [ ]:
dist.new_group()

In [ ]:
class _P2P:
    def _recv_metadata(self, src_rank, parallel_context, parallel_mode):
        group = parallel_context.get_group(parallel_mode)
        
        dtype = torch.tensor(0)
        dist.recv(dtype, src=src_rank, group=group)
        dtype = ID_TO_DTYPE[dtype]
        
        requires_grad = torch.tensor(0)
        dist.recv(requires_grad, src=src_rank, group=group)
        requires_grad = True if requires_grad = 1 else 0
        
        shape = torch.tensor(0)
        dist.recv(shape, src=src_rank, group=group)
        
        return dtype, requires_grad, shape
        
    def recv(self, src_rank, parallel_context, parallel_mode):
        group = parallel_context.get_group(parallel_mode)
        
        dtype, requires_grad, shape = self._recv_metadata(src_rank, parallel_context, parallel_mode)
        data = torch.zeros(size=shape, requires_grad=requires_grad, dtype=dtype)
        dist.recv(data, src=src_rank, group=group)
        
        return data

In [13]:
def recv(src_rank, dst_rank, parallel_context, parallel_mode):
    rank = parallel_context.get_local_rank(parallel_model.PIPELINE)
    
    if dst_rank == rank:
        return _P2P().recv(src_rank, parallel_context, parallel_mode)

In [ ]:
minimize communication
maximize storage
minimize flops

In [ ]:
step 1: raw_data + metadata = package
step 2: invoke rpc call, along with the local data
step 3: destination node invoked the call, 
step 4: 

In [ ]:
shape, dtype, requires_grad

In [ ]:
step 1: mask targets
step 2: calculate the local predicted logits
step 3: calcualte the global predicted logits
step 4: log(...)
step 5: calculate the loss

In [15]:
n_microbatches = 4
n_partitions = 3

In [16]:
n_clock_cycles = n_microbatches+n_partitions-1

In [17]:
n_clock_cycles

6

In [18]:
for clock_idx in range(n_clock_cycles):
    start_partition_idx = max(clock_idx+1-n_microbatches, 0)
    end_partition_idx = min(clock_idx+1, n_partitions)
    
    xs = []
    for partition_idx in range(start_partition_idx, end_partition_idx):
        microbatch_idx = clock_idx - partition_idx
        xs.append((microbatch_idx, partition_idx))
    
    print(xs)

[(0, 0)]
[(1, 0), (0, 1)]
[(2, 0), (1, 1), (0, 2)]
[(3, 0), (2, 1), (1, 2)]
[(3, 1), (2, 2)]
[(3, 2)]


In [ ]:
step 1: determine global rank
step 2: resize embedding
step 3: parallelize embedding, linear, attn, layer norm
step 4: resize lm_head

In [ ]:
step 1: normalize the loss / n_epochs
step 2: calculate the gradients with respect to the normalzied loss
step 3: accumulate the grads
step 4: if current_epoch == n_epoch, update, otherwise, repeat step 1

In [ ]:
class TestFruit:
    def setup_method(self):
        self.fruit = Fruit("x")
    
    def test_fruit(self):
        assert self.fruit.name == "x"
    
    def teardown_method(self):
        del self.fruit

In [ ]:
embed = model.embed
mlp0 = model.blocks[0].mlp
ln0 = model.blocks[0].ln2
unembed = model.unembed
ln_final = model.ln_final

In [ ]:
name_tokens = model.to_tokens(names)
name_embeddings = embed(name_tokens)

In [ ]:
resid_after_mlp0 = name_embeddings + mlp0(ln0(name_embeddings))

In [ ]:
W_OV = model.W_V[0, 1] @ model.W_O[0, 1]

In [ ]:
step 1: prob[0] = sigmoid(logit0 - logit1)
step 2: logit0 = resid @ W_U[0], logit1 = resid @ W_U[1]
step 3: logit0 - logit1 = resid @ (W_U[0] - W_U[1])

In [ ]:
_, clean_cache = model.run_with_cache(clean_tokens)
_, corrupted_cache = model.run_with_cache(corrupted_tokens)

In [19]:
def patch_head(acts, hook, clean_cache, corrupted_cache, target_head):
    target_layer_idx, target_head_idx = target_head
    
    if target_layer_idx == hook.layer():
        acts[:, :, target_head_idx] = corrupted_cache[hook.name][:, :, target_head_idx]
    else:
        acts = clean_cache[hook.name]
    
    return acts

In [20]:
from transformer_lens.utils import get_act_name

In [21]:
from functools import partial

In [ ]:
results = torch.zeros(n_layers, n_heads)

for layer_idx in range(n_layers):
    for head_idx in range(n_heads):
        model.reset_hooks()
        hook_name = get_act_name("z", layer_idx)
        hook_func = partial(
            patch_head,
            clean_cache=clean_cache,
            corrupted_cache=corrupted_cache,
            target_head=(layer_idx, head_idx)
        )
        
        model.add_hook(hook_name, hook_func)
        patched_logits, _ = model.run_with_cache(clean_tokens)
        results[layer_idx, head_idx] = compute_ioi_metric(patched_logits)

In [22]:
from contextlib import contextmanager

In [23]:
@contextmanager
def first_context():
    print("your mom")
    
    yield
    
    print("xxx")

In [25]:
from torch.distributed.rpc import RRef

In [26]:
class Observer:
    def __init__(self, env):
        self.id = RRef(self)
        self.env = env
    
    def run_episode(self, agent_rref):
        state, _ = self.env.reset()
        for _ in range(69):
            action = rpc.rpc_sync(
                to=agent_rref.owner(),
                func=agent_rref.rpc_sync().select_action,
                args=(self.id, state)
            )
            
            state, reward, done, _ = env.step(action)
            if done:
                break

In [27]:
import threading

In [ ]:
thread = threading.Thread(
    target=print_numbers
)

In [ ]:
_, cache = model.run_with_cache(
    tokens
)

In [ ]:
mlp_names = [get_act_name("mlp_out", layer_idx) for layer_idx in range(n_layers)]
act_names = [get_act_name("attn_out", layer_idx), for layer_idx in range(n_layers)]

components = [
    "embed", "pos_embed",
    mlp_names,
    act_name
]

In [ ]:
theirs = linear_probe[..., 1]
mine = linear_probe[..., 2]

mine_vs_theirs = mine - theirs

In [ ]:
_, cache = model.run_with_cache(
    board_history
)

In [28]:
from einops import einsum

In [ ]:
attn_contributions = []

for layer_idx in range(n_layers):
    attn_contributions.append(einsum(
        cache["attn_out", layer_idx],
        mine_vs_theirs
    ))

In [ ]:
embed = model.embed
mlp0 = model.blocks[0].mlp
ln0 = model.blocks[0].ln2
unembed = model.unembed
ln_final = model.ln_final

In [ ]:
name_tokens = model.to_tokens(names, prepend_bos=False)
name_embeddings = embed(name_tokens)

In [ ]:
resid_after_mlp0 = name_embeddings + mlp0(ln0(name_embeddings))

In [ ]:
results = torch.zeros

In [29]:
def probability_scores(image_embedding, text_embedding):
    image_norm = image_embedding.norm(dim=-1)
    image_embedding /= image_norm
    
    text_norm = text_embedding.norm(dim=-1)
    text_embedding /= text_norm
    
    similarities = image_embedding @ text_embedding.T
    probs = F.softmax(similarities, dim=-1)
    return probs

In [ ]:
embed = model.embed
mlp0 = model.blocks[0].mlp
ln01 = model.blocks[0].ln2

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
text_embeddings = embed(tokens)

In [ ]:
resid_after_mlp0 = text_embeddings + mlp0(ln01(text_embeddings))

In [ ]:
results = torch.zeros(n_layers, n_heads)

In [ ]:
name_tokens = model.to_tokens(names)
name_embeddings = model.embed(name_tokens)


In [ ]:
for layer_idx in range(n_layers-1):
    for head_idx in range(n_heads):
        W_OV = model.W_V[layer_idx, head_idx] @ model.W_O[layer_idx, head_idx]
        

In [ ]:
name_tokens = model.to_tokens(names)
name_embeddings = model.embed(name_tokens)

mlp0 = model.blocks[0].mlp
ln2 = model.blocks[0].ln2

resid_after_mlp0 = name_embeddings + mlp0(ln2(name_emebddings))

In [ ]:
for layer_idx in range(n_layer-1):
    for head_idx in range(n_heads):
        W_OV = model.W_V[layer_idx, head_idx] @ W_O[layer_idx, head_idx]
        output = resid_after_mlp0 = W_OV
        outpu

In [ ]:
population decoding, stimulus evoked, motor imagery, 